In [13]:
# Read dataset
import numpy as np
import pandas as pd

df = pd.read_csv('data/selected_data.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.iloc[:,:-1]

In [14]:
# Standarize the data
from sklearn.preprocessing import StandardScaler

cols = df.columns.values
df = pd.DataFrame(StandardScaler().fit_transform(df), columns=cols)

In [15]:
# Calculate reconstruction errors
from sklearn.decomposition import PCA
from models.model import ReconstructionErrorModel

model = ReconstructionErrorModel(df, model=PCA(n_components=0.95))
errors = model.predict(df)

In [16]:
# Classify anomalies
threshold = np.std(errors) * 3
df['target'] = 0


for anomaly in np.where(errors > threshold):    
    #df['target'][anomaly] = 1
    df.at[anomaly, 'target'] = 1

df['target'].value_counts()

0    9977
1      37
Name: target, dtype: int64